In [236]:
import openpyxl
import polars as pl
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
import re



In [237]:
IMPRIMIR = False

def imprimir(mensaje: str, parametros=None, collect_lazy_frame=False, **kwargs):
    if not IMPRIMIR:
        return
    # Normaliza la fuente de valores: dict explícito o kwargs
    values = {}
    if isinstance(parametros, dict):
        values.update(parametros)
    values.update(kwargs)

    # Soporte opcional para LazyFrame
    if collect_lazy_frame:
        for k, v in list(values.items()):
            if hasattr(v, "collect"):
                values[k] = v.collect()

    print(mensaje.format(**values))

In [238]:
ruta = "data/opiniones.xlsx"
workbook = openpyxl.load_workbook(ruta, read_only=True)
hojas = workbook.sheetnames


In [239]:

def obtener_nombre_hoja(sheet_index: int) -> str:
    """Obtiene el nombre de la hoja dado su índice."""
    nombre_hoja = hojas[sheet_index].lower().replace(" ", "_")
    imprimir(f"Nombre de la hoja: {nombre_hoja}")
    return nombre_hoja


def calificaciones(calificacion:str) -> int:
    """Convierte la calificación de string a int."""
    calificacion_map = {
        "Pésimo": 1,
        "Malo": 2,
        "Regular": 3,
        "Muy bueno": 4,
        "Excelente": 5
    }
    return calificacion_map.get(calificacion, 0)


def tratar_columna(col:str) -> str:
    """Trata el nombre de una columna para que sea válido en polars."""
    col = col.strip().lower().replace(" ", "_").replace("-", "_")
    return col


def añadir_columna_lugar_turistico(iteracion: int) -> pl.LazyFrame:
    df_excel = pl.read_excel(ruta, sheet_id=iteracion+1).lazy()
    nombre_hoja = obtener_nombre_hoja(iteracion)
    
    df_excel = (
        df_excel
        .with_columns([
            pl.lit(nombre_hoja).alias("lugar_turistico"),
            pl.col("Calificación").map_elements(calificaciones).alias("calificacion_numerica"),
        ])
    )

    df_excel = df_excel.rename({col: tratar_columna(col) for col in df_excel.collect_schema().names()})
    return df_excel


def juntar_df(lista_df: list[pl.LazyFrame]) -> pl.LazyFrame:
    nuevo_df_excel = pl.concat(lista_df)
    return nuevo_df_excel




# Limpiezsa y tratamiento del dataset

In [240]:
lista_df: list[pl.LazyFrame] = []
for i, hoja in enumerate(hojas):
    lista_df.append(añadir_columna_lugar_turistico(i))

df_excel = juntar_df(lista_df)


In [241]:
# Guardar el DataFrame en un parquet
ruta_parquet = "data/opiniones_lugar_turistico.parquet"

df_excel = df_excel.collect()
df_excel.write_parquet(ruta_parquet)

imprimir("Archivo guardado en: {ruta_parquet}", {"ruta_parquet": ruta_parquet})

In [242]:
df_reseñas = pl.scan_parquet(ruta_parquet)


imprimir(f"{df_reseñas.collect()['opinión'][:5]}")

In [243]:

def estadisticos(lf: pl.LazyFrame) -> pl.DataFrame:
    stats = (
        lf
        .group_by("lugar_turistico")
        .agg([
            # Promedio de calificación
            pl.col("calificacion_numerica").mean().alias("promedio_calificacion"),
            # Desviación estándar de calificación
            pl.col("calificacion_numerica").std().alias("desviacion_estandar_calificacion"),

            # Longitud promedio y std de las opiniones
            pl.col("opinión").str.split(" ").list.len().mean().alias("longitud_promedio_opiniones"),

            # Desviación estándar de la longitud de las opiniones
            pl.col("opinión").str.split(" ").list.len().std().alias("desviacion_estandar_longitud_opiniones"),

            # Edad promedio
            pl.col("edad").mean().alias("edad_promedio"),
            # Desviación estándar de la edad
            pl.col("edad").std().alias("desviacion_estandar_edad"),

            # % nacional como aproximación a (d) (aun no jala bien)
            # TODO: revisar esta parte
            (
                (pl.col("nacional_ó_internacional") == "nacional")
                .cast(pl.Int8)
                .mean()
                .alias("porcentaje_nacional")
            ),
        ])
    )

    return stats.collect()


imprimir(f"{estadisticos(df_reseñas)}")

# CONTRUCCION DEL VOCABULARIO, DISTRIBUCIÓN FRECUENCIAS

In [244]:
def tokenizador(texto: str) -> list[str]:
    sos = "<SOS>"
    eos = "<EOS>"
    stop_words = set(stopwords.words("spanish"))
    t = TweetTokenizer()

    texto  =   re.sub(r'^.', sos, texto)
    texto  =   re.sub(r'.$', eos, texto)
    texto_lower = t.tokenize(texto.lower().strip())
    if not texto_lower:
        return [sos, eos]
    
    texto_tokenizado = [token  for token in texto_lower if token not in stop_words]
    return texto_tokenizado



df_reseñas = (
    df_reseñas
    .with_columns([
        pl.col("opinión").map_elements(tokenizador).alias("opinión_tokenizada")
    ])
)


#imprimir(f"{df_reseñas.collect()['opinión_tokenizada'][0]}")

# Distribución de frecuenciaS

In [245]:
distribucion_frecuencias = (
    df_reseñas
    .explode("opinión_tokenizada")
    .group_by("opinión_tokenizada")
    .agg([
        pl.len().alias("frecuencia")
    ])
    .sort("frecuencia", descending=True)
)

#imprimir(f"{distribucion_frecuencias.collect()}")

# VOCABILARIO

In [246]:
vocabulario = (
    distribucion_frecuencias
    .select(pl.col("opinión_tokenizada"))
)


print(f"{vocabulario.collect()}")

shape: (23_099, 1)
┌────────────────────┐
│ opinión_tokenizada │
│ ---                │
│ str                │
╞════════════════════╡
│ .                  │
│ ,                  │
│ the                │
│ <eos>              │
│ <sos>              │
│ …                  │
│ doorways           │
│ detonaron          │
│ íntimas            │
│ foyers             │
│ nearlly            │
└────────────────────┘
